# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
#Use the get_files function provided above to get a list of all song JSON files in data/song_data
song_files = get_files('data/song_data')

In [5]:
#Select the first song in this list
filepath =song_files[1]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json


In [6]:
#Read the song file and view the data
df = pd.read_json(filepath,lines=True)
print(df)

            artist_id  artist_latitude artist_location  artist_longitude  \
0  ARNTLGG11E2835DDB9              NaN                               NaN   

  artist_name   duration  num_songs             song_id  \
0         Clp  266.39628          1  SOUDSGM12AC9618304   

                               title  year  
0  Insatiable (Instrumental Version)     0  


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
#song_id=df['song_id'].values[0]
#title=df['title'].values[0]
#artist_id=df['artist_id'].values[0]
#year=int(df['year'].values[0])
#duration=df['duration'].values[0]
song_data = tuple(df[['song_id','title','artist_id','year','duration']].values[0])
#song_data =[song_id,title,artist_id,year,duration]
song_data

('SOUDSGM12AC9618304',
 'Insatiable (Instrumental Version)',
 'ARNTLGG11E2835DDB9',
 0,
 266.39628)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
#artist_data = [df['artist_id'].values[0],df['artist_name'].values[0],df['artist_location'].values[0],df['artist_latitude'].values[0],df['artist_longitude'].values[0]]
artist_data =tuple(df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0])
artist_data

('ARNTLGG11E2835DDB9', 'Clp', '', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files =  get_files('data/log_data')

In [12]:
filepath = log_files[1]

In [13]:
df = pd.read_json(filepath,lines=True)
#print(log_files[1])
print(df.head(5))

                      artist       auth firstName gender  itemInSession  \
0                   Frumpies  Logged In  Anabelle      F              0   
1  Kenny G with Peabo Bryson  Logged In  Anabelle      F              1   
2                Biffy Clyro  Logged In  Anabelle      F              2   
3                       None  Logged In      Lily      F              0   
4                        HIM  Logged In      Lily      F              1   

  lastName     length level                                     location  \
0  Simpson  134.47791  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
1  Simpson  264.75057  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
2  Simpson  189.83138  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
3    Burns        NaN  free        New York-Newark-Jersey City, NY-NJ-PA   
4    Burns  212.06159  free        New York-Newark-Jersey City, NY-NJ-PA   

  method      page  registration  sessionId                            song  \
0    PUT  Nex

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = pd.read_json(filepath,lines=True)

#Filter records by NextSong action
df=df[df['page']=='NextSong']

#Convert the ts timestamp column to datetime
#df['ts']=pd.to_datetime(df['ts'], unit='ms')

#Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
#time_data=[df['ts'],df['ts'].dt.hour,df['ts'].dt.day,df['ts'].dt.week,df['ts'].dt.year,df['ts'].dt.weekday]
#df.head()
#Specify labels for these columns and set to column_labels
#column_labels=['Timestamp','Hours','Day','Week','Year','Weekday']
#Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe
#time_df=pd.concat(time_data,axis=1)
#time_df.columns=column_labels
#print(time_df.head())



In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = [t.dt.time,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday]
column_labels = ['Time','Hours','Day','Week','Month','Year','Weekday']

In [102]:
time_df = pd.concat(time_data,axis=1)
time_df.columns=column_labels
time_df.head()

,Time,Hours,Day,Week,Month,Year,Weekday
0,02:33:56.796000,2,11,45,11,2018,6
1,02:36:10.796000,2,11,45,11,2018,6
2,02:40:34.796000,2,11,45,11,2018,6
3,04:34:01.796000,4,11,45,11,2018,6
4,04:36:13.796000,4,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [103]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    #print(type(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [104]:
usr_series=[df['userId'],df['firstName'],df['lastName'],df['gender'],df['level']]
#user_df = df[['userId','firstName','lastName','gender','level']]
#print(type(user_df))
user_df = pd.concat(usr_series,axis=1)
#print(type(user_df))
#print(user_df.head())

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [108]:
for i, row in user_df.iterrows():
    if not list(row)[0]=="":
        cur.execute(user_table_insert, list(row))
        conn.commit()
    else:
        print("empty user_id.Record will be ignored..")


empty user_id.Record will be ignored..
empty user_id.Record will be ignored..
empty user_id.Record will be ignored..


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [118]:
for index, row in df.iterrows():
    #print(song_select)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
  
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    #print(row)
    # insert songplay record
    #print(row.ts)
    time=str(pd.to_datetime(row.ts, unit='ms').time())
    #print("time "+ str(time))
    songplay_data = (time,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    print(type(songid))    
    if not songid==None:
        print(songid+str(songid))
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    else:
        print("No songid found for "+str(row.song) +" "+ str(row.artist)+" "+str(row.length))
print("--------------------------")
#cur.execute('SELECT * FROM songplays')
#res=cur.fetchall()
#print(res)

<class 'NoneType'>
No songid found for Fuck Kitty Frumpies 134.47791
<class 'NoneType'>
No songid found for By The Time This Night Is Over Kenny G with Peabo Bryson 264.75057
<class 'NoneType'>
No songid found for God & Satan Biffy Clyro 189.83138
<class 'NoneType'>
No songid found for None None nan
<class 'NoneType'>
No songid found for Beautiful HIM 212.06159
<class 'NoneType'>
No songid found for Supreme Balloon Matmos 1449.11628
<class 'NoneType'>
No songid found for The One Gary Allan 259.83955
<class 'NoneType'>
No songid found for Five Roses Miracle Fortress 200.9073
<class 'NoneType'>
No songid found for Cuentale Don Omar 261.35465
<class 'NoneType'>
No songid found for D'Evils Jay-Z 212.27057
<class 'NoneType'>
No songid found for Easily (Album Version) Red Hot Chili Peppers 231.33995
<class 'NoneType'>
No songid found for None None nan
<class 'NoneType'>
No songid found for Rebels of the Sacred Heart Flogging Molly 361.9522
<class 'NoneType'>
No songid found for Now_ Right No

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.